In [ ]:
#!pip install seqeval==0.0.5
!pip install -U seqeval

  Using cached seqeval-1.2.2-py3-none-any.whl
  Attempting uninstall: seqeval
    Found existing installation: seqeval 0.0.5
    Uninstalling seqeval-0.0.5:
      Successfully uninstalled seqeval-0.0.5


In [ ]:
import os
import openai
import json

In [ ]:
import pandas as pd

df = pd.read_json("hf://datasets/yixuantt/FinEntity/FinEntity.json")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
json_str = df.to_json(orient='records')

In [ ]:
raw = json.loads(json_str)

In [ ]:
from google.colab import userdata
openai_api = userdata.get('openai_api')

In [ ]:
# raw = json.load(open('./data/FinEntity.json'))
raw = raw[:200]
openai.api_key = openai_api

sys_prompt = "Discard all the previous instructions. Behave like you are an expert entity recognizer and sentiment classifier. "
user_prompt = """
Identify the entities which are companies or organizations from the following content and classify the sentiment of the corresponding entities into ‘Neutral’, ‘Positive’, or ‘Negative’ classes.
Considering every sentence as a String in python, provide the entities with the start and end index to mark the boundaries of it including spaces and punctuation using zero-based indexing.
Do not give explanations for the sentiment. In the output,Tag means sentiment; value means entity name. If no entity is found in the sentence, the response should be empty.
The sentence: "Other U.S. companies have made similar moves, including social media site Reddit Inc and Mobileye, the self-driving car unit of Intel Corp <INTC.O>. "
"""
assist_prompt = """{"start": 74, "end": 84, "value": "Reddit Inc", "tag": "Neutral"}\n{"start": 128, "end": 138, "value": "Intel Corp", "tag": "Neutral"}
"""
user_prompt2 = """Kellogg <K.N>, however, based the corporate headquarters for its largest business, snacks, in Chicago after announcing a split into three independent companies this summer. [nL4N2Y822D]
"""
assist_prompt2 = """{"start": 4, "end": 11, "value": "Kellogg", "tag": "Neutral"}
"""

user_prompt3 = """Rival Oracle <ORCL.N> says in a statement on its website it has withdrawn all products, services and support for Russian and Belarusian companies, subsidiaries and partners. An Oracle spokesperson declined further comment.
"""

assist_prompt3= """{"start": 183, "end": 177, "value": "Oracle", "tag": "Neutral"}\n{"start": 6, "end": 12, "value": "Oracle", "tag": "Neutral"}
"""


In [ ]:
def subset(alist, idxs):
    sub_list = []
    for idx in idxs:
        sub_list.append(alist[idx])

    return sub_list

import time
result_list=[]
compare_list=[]
for item in raw:
    sentence = item['content']
    #sentence ='Nearly all major S&P 500 sectors are red, with materials <.SPLRCM> and communications services <.SPLRCL> taking the biggest hits. Staples <.SPLRCS> and healthcare <.SPXHC> are posting small gains.'
    try:
        rsp = openai.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt},
                {"role": "assistant", "content": assist_prompt},
                {"role": "user", "content": user_prompt2},
                {"role": "assistant", "content": assist_prompt2},
                {"role": "user", "content": user_prompt3},
                {"role": "assistant", "content": assist_prompt3},
                {"role": "user", "content": sentence},
            ],
            temperature=0.0,
        )
    except:
        print("retry request")
        time.sleep(0.5)
        rsp = openai.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[{"role": "user", "content": sentence},],temperature=0.0,)
    else:
        result_dict = {}
        result_dict['content'] = sentence
        if len(rsp.choices)==0:
            result_dict['annotations'] = []
            result_list.append(result_dict)
            compare_list.append(item)
            continue
        choice = rsp.choices[0]
        message = choice.message
        res_str = message.content
        res_str = res_str.split('\n')
        anno_list = []
        if len(res_str)==0:
            result_dict['annotations'] = []
            result_list.append(result_dict)
            compare_list.append(item)
            continue
        for res in res_str:
            index_left = res.find('{')
            index_right = res.find('}')
            if index_right == -1 or index_left == -1:
                continue
            res = res[index_left:index_right+1]
            sub_json = json.loads(res)
            anno_list.append(sub_json)
        result_dict['annotations'] = anno_list
        result_list.append(result_dict)
        compare_list.append(item)

In [ ]:
#Correcting start and end tags
for i,item in enumerate(result_list):
    text = item['content']
    annos = item['annotations']
    try:
      sorted_annos = sorted(annos, key=lambda x: x['start'])
    except:
      continue
    value_list = []
    start_list = []
    drop_list = []
    for indx,sub_annos in enumerate(sorted_annos):
        value = sub_annos['value']
        if value not in value_list:
            start = text.find(value)
        else:
            index_list = []
            for j,v in enumerate(value_list):
                if v==value:
                    index_list.append(j)
            sub_start = subset(start_list,index_list)
            last_start = max(sub_start)
            start = text.find(value,last_start+1)
        sub_annos['start'] = start
        sub_annos['end'] = start + len(value)
        value_list.append(value)
        start_list.append(start)

In [ ]:
with open('/content/open_ai.json', 'wt') as f:
    print(json.dumps(result_list), file=f)
print("COMPLETE")

COMPLETE


In [ ]:
print(len(compare_list))
print(len(result_list))

200
200


In [ ]:
for idx,item in enumerate(result_list):
    annos = item['annotations']
    drop_list=[]
    try:
      item['annotations'] = [x for x in annos if x['start']>0]
    except:
      item['annotations'] = []

In [ ]:
for idx,item in enumerate(result_list):
    annos = item['annotations']
    for a in annos:
        if a['start']<0:
            print(item)

In [ ]:
#result_list = json.load(open('./data/open_ai.json'))
for example in result_list:
    for annotation in example['annotations']:
        #We expect the key of label to be label but the data has tag
        annotation['label'] = annotation['tag']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Courses/Spring 2025/COS 484/final project/chae')

In [ ]:
from sequence_aligner.labelset import LabelSet
from sequence_aligner.dataset import TrainingDatasetCRF
from sequence_aligner.containers import TraingingBatch
from transformers import BertTokenizerFast

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('yiyanghkust/finbert-pretrain')
label_set = LabelSet(labels=["Neutral", "Positive", "Negative"])  # label in this dataset

dataset = TrainingDatasetCRF(data=compare_list, tokenizer=tokenizer, label_set=label_set,tokens_per_batch = 128)
dataset_openai = TrainingDatasetCRF(data=result_list, tokenizer=tokenizer, label_set=label_set,tokens_per_batch = 128)

In [ ]:
from seqeval.metrics import f1_score
from seqeval.metrics import precision_score
from seqeval.metrics import accuracy_score
from seqeval.metrics import recall_score
from seqeval.metrics import classification_report
from util.process import ids_to_labels,Metrics,Metrics_e
from seqeval.scheme import BILOU

In [ ]:
label_list=[]
pred_label_list=[]
for i in range(len(dataset)):
    sub_list=[]
    pred_sub_list=[]
    for m in dataset[i].labels:
        if m == -1:
            continue
        else:
            sub_list.append(label_set.ids_to_label[m])
    for n in dataset_openai[i].labels:
        if n == -1:
            continue
        else:
            if n == None:
                n = 0
            pred_sub_list.append(label_set.ids_to_label[n])
    label_list.append(sub_list)
    pred_label_list.append(pred_sub_list)
report=classification_report(label_list, pred_label_list, mode='strict', scheme=BILOU)
print(report)

              precision    recall  f1-score   support

    Negative       0.70      0.49      0.58        63
     Neutral       0.76      0.66      0.71       139
    Positive       0.86      0.66      0.75       225

   micro avg       0.80      0.64      0.71       427
   macro avg       0.78      0.61      0.68       427
weighted avg       0.81      0.64      0.71       427

